<a href="https://colab.research.google.com/github/gayaniindunil/facegeneration_GAN/blob/master/simple_image_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#this program is created to generate face images using a unconditional GAN.
#USED IMAGE DATASET  -> Person images from Natural Images dataset
#number of images -> 986 images
#original image size -> 256 X 265

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#papers : 
# 1.https://arxiv.org/pdf/1511.06434.pdf%C3%AF%C2%BC%E2%80%B0%C3%A3%E2%82%AC%E2%80%98
# 2. 

#Refered github repositories : 
# 1. art-DCGAN : https://github.com/robbiebarrat/art-DCGAN
# 2. https://github.com/mitchelljy/DCGAN-Keras/blob/master/DCGAN.py

In [0]:
from keras.models import Sequential, Model, load_model
from keras.layers import UpSampling2D, Conv2D, Activation, BatchNormalization, Reshape, Dense, Input, LeakyReLU, Dropout, Flatten, ZeroPadding2D
from keras.optimizers import Adam
from keras.preprocessing import image
import numpy as np
from PIL import Image
import os
import glob
# from scipy.misc import *
import imageio

Using TensorFlow backend.


In [0]:
root_dir = "/content/drive/My Drive/Colab_Notebooks/image_generation"


Mounted at /content/drive


In [0]:
# model parameters
data_dir = root_dir + '/datasets/person/*.jpg'

output_save_dir = root_dir + '/Explore_GANs/DCGAN/output'
model_save_name = 'DCGAN.h5'

save_path = output_save_dir + "/models"
discriminator_path = save_path + "/discrim.h5"
generator_path = save_path + "/generat.h5"
output_img_save_dir = root_dir + '/Explore_GANs/DCGAN/output_imgs'

output_directory = output_img_save_dir

In [0]:
#hyper parameters
img_size = (256, 256)
channels = 3
kernel_size = 3
upsample_layers = 5
starting_filters = 64

In [0]:
def load_imgs1(data_dir):
  images = os.listdir(data_dir)

  for img_path in images:
    img = Image.open(data_dir + '/' + img_path)
    
    w,h = img.size
    rescale_ratio = 256 / min(w,h)
    img = img.resize((int(rescale_ratio*w),int(rescale_ratio*h)), Image.LANCZOS)
    # display(img)

In [0]:
def load_imgs(image_path):
  X_train = []
  for i in glob.glob(image_path):
    img = Image.open(i)
    img = img.resize((256,256))
    img = np.array(img)
    X_train.append(img)
  
  X_train = np.array(X_train)
  return  X_train


In [0]:
x = [659, 976,  16, 119, 721, 148]

imgs = []
for img in x:
  i = (a[img])
  w,h = i.shape[0:2]
  print(w,h)
#   rescale_ratio = 256 / min(w,h)
#   img = i.resize((int(rescale_ratio*w),int(rescale_ratio*h)), Image.LANCZOS)
#   # display(img)
#   # Rescale images from -1 to 1
#   i = (i.astype(np.float32) - 127.5) / 127.5
#   display(i)
#   imgs.append(i)
# imgs = np.array(imgs)

NameError: ignored

In [0]:
def build_generator():
  noise_shape = (100,)

  # This block of code can be a little daunting, but essentially it automatically calculates the required starting
  # array size that will be correctly upscaled to our desired image size.
  #
  # We have 5 Upsample2D layers which each double the images width and height, so we can determine the starting
  # x size by taking (x / 2^upsample_count) So for our target image size, 256x192, we do the following:
  # x = (192 / 2^5), y = (256 / 2^5) [x and y are reversed within the model]
  # We also need a 3rd dimension which is chosen relatively arbitrarily, in this case it's 64.
  model = Sequential()
  model.add(Dense(starting_filters * (img_size[0] // (2 ** upsample_layers))  *  (img_size[1] // (2 ** upsample_layers)),
            activation="relu", input_shape=noise_shape))
  model.add(Reshape(((img_size[0] // (2 ** upsample_layers)),
                      (img_size[1] // (2 ** upsample_layers)),
                      starting_filters)))
  model.add(BatchNormalization(momentum=0.8))

  model.add(UpSampling2D())  # 6x8 -> 12x16
  model.add(Conv2D(1024, kernel_size=kernel_size, padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(momentum=0.8))

  model.add(UpSampling2D())  # 12x16 -> 24x32
  model.add(Conv2D(512, kernel_size=kernel_size, padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(momentum=0.8))

  model.add(UpSampling2D())  # 24x32 -> 48x64
  model.add(Conv2D(256, kernel_size=kernel_size, padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(momentum=0.8))

  model.add(UpSampling2D())  # 48x64 -> 96x128
  model.add(Conv2D(128, kernel_size=kernel_size, padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(momentum=0.8))

  model.add(UpSampling2D())  # 96x128 -> 192x256
  model.add(Conv2D(64, kernel_size=kernel_size, padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Conv2D(32, kernel_size=kernel_size, padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Conv2D(channels, kernel_size=kernel_size, padding="same"))
  model.add(Activation("tanh"))

  model.summary()

  noise = Input(shape=noise_shape)
  img = model(noise)

  return Model(noise, img)

In [0]:
def build_discriminator():
  img_shape = (img_size[0], img_size[1], channels)

  model = Sequential()

  model.add(Conv2D(32, kernel_size=kernel_size, strides=2, input_shape=img_shape, padding="same"))  # 192x256 -> 96x128
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, kernel_size=kernel_size, strides=2, padding="same"))  # 96x128 -> 48x64
  model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Conv2D(128, kernel_size=kernel_size, strides=2, padding="same"))  # 48x64 -> 24x32
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Conv2D(256, kernel_size=kernel_size, strides=1, padding="same"))  # 24x32 -> 12x16
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))

  model.add(Conv2D(512, kernel_size=kernel_size, strides=1, padding="same"))  # 12x16 -> 6x8
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))

  model.summary()

  img = Input(shape=img_shape)
  validity = model(img)

  return Model(img, validity)


In [0]:
def build_gan():
  optimizer = Adam(0.0002, 0.5)

  # See if the specified model paths exist, if they don't then we start training new models

  if os.path.exists(discriminator_path) and os.path.exists(generator_path):
      discriminator = load_model(discriminator_path)
      generator = load_model(generator_path)
      print("Loaded models...")
  else:
      discriminator = build_discriminator()
      discriminator.compile(loss='binary_crossentropy',
                                  optimizer=optimizer,
                                  metrics=['accuracy'])

      generator = build_generator()
      generator.compile(loss='binary_crossentropy', optimizer=optimizer)

  # These next few lines setup the training for the GAN model
  z = Input(shape=(100,))
  img = generator(z)

  discriminator.trainable = False

  valid = discriminator(img)

  combined = Model(z, valid)
  combined.compile(loss='binary_crossentropy', optimizer=optimizer)

  return (generator,discriminator,combined)

In [0]:
def train(epochs, image_path, batch_size=12, save_interval=20):
  generator,discriminator,combined = build_gan()
  
  X_train = load_imgs(image_path)
  print("Training Data Shape: ", X_train.shape)

  half_batch = batch_size // 2

  for epoch in range(epochs):
    # Train Generator
    noise = np.random.normal(0, 1, (batch_size, 100))
    g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))

    # Train Discriminator
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    imgs = []
    for img in idx:
      i = (X_train[img])
      # Rescale images from -1 to 1
      i = (i.astype(np.float32) - 127.5) / 127.5
      imgs.append(i)
    imgs = np.array(imgs)

    # Sample noise and generate a half batch of new images
    noise = np.random.normal(0, 1, (half_batch, 100))
    gen_imgs = generator.predict(noise)

    print(idx)
    print("epoch: images_shape: gen_images.shape: =", epoch, imgs.shape, gen_imgs.shape)

    # Train the discriminator (real classified as ones and generated as zeros)
    d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch,1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Print progress
    print(f"{epoch} [D loss: {d_loss[0]} | D Accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

    # If at save interval => save generated image samples, save model files
    if epoch % (save_interval) == 0:

      save_imgs(epoch,generator)

      if not os.path.exists(save_path):
          os.makedirs(save_path)
      discriminator.save(save_path + "/discrim.h5")
      generator.save(save_path + "/generat.h5")

In [0]:
def gene_imgs(count,generator):
  # Generate images from the currently loaded model
  noise = np.random.normal(0, 1, (count, 100))
  return generator.predict(noise)


In [0]:
def save_imgs(epoch, generator):
  r, c = 5, 5

  # Generates r*c images from the model, saves them individually and as a gallery

  imgs = gene_imgs(r*c,generator)
  imgs = 0.5 * imgs + 0.5

  print(imgs.shape)

  for i, img_array in enumerate(imgs):
      path = f"{output_directory}/generated_{img_size[0]}x{img_size[1]}"
      if not os.path.exists(path):
          os.makedirs(path)
      imageio.imwrite(path + f"/{epoch}_{i}_4.png", img_array)

  nindex, height, width, intensity = imgs.shape
  nrows = nindex // c
  assert nindex == nrows * c
  # want result.shape = (height*nrows, width*ncols, intensity)
  gallery = (imgs.reshape(nrows, c, height, width, intensity)
            .swapaxes(1, 2)
            .reshape(height * nrows, width * c, intensity))

  path = f"{output_directory}/gallery_generated_{img_size[0]}x{img_size[1]}"
  if not os.path.exists(path):
      os.makedirs(path)
  imageio.imwrite(path + f"/{epoch}_4.png", gallery)

In [0]:
def generate_imgs(count, threshold, modifier):
  generator,discriminator,combined = build_gan()

  # Generates (count) images from the model ensuring the discriminator scores them between the threshold values
  # and saves them

  imgs = []
  for i in range(count):
      score = [0]
      while not(threshold[0] < score[0] < threshold[1]):
          img = gene_imgs(1,generator)
          score = discriminator.predict(img)
      print("Image found: ", score[0])
      imgs.append(img)

  imgs = np.asarray(imgs).squeeze()
  imgs = 0.5 * imgs + 0.5

  print(imgs.shape)
  for i, img_array in enumerate(imgs):
      path = f"{output_directory}/generated_{threshold[0]}_{threshold[1]}"
      if not os.path.exists(path):
          os.makedirs(path)
      imageio.imwrite(path + f"/{modifier}_{i}.png", img_array)

In [0]:
train(epochs=1001, image_path=data_dir)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Loaded models...
Training Data Shape:  (986, 256, 256, 3)
[701 443  35 541 710 847]
epoch: images_shape: gen_images.shape: = 0 (6, 256, 256, 3) (6, 256, 256, 3)


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 2.8727383613586426 | D Accuracy: 50.0] [G loss: 10.774041175842285]
(25, 256, 256, 3)


[776 521 817 777 536 796]
epoch: images_shape: gen_images.shape: = 1 (6, 256, 256, 3) (6, 256, 256, 3)
1 [D loss: 1.4997305870056152 | D Accuracy: 66.66666269302368] [G loss: 5.015303134918213]
[596 591 363  31 344 272]
epoch: images_shape: gen_images.shape: = 2 (6, 256, 256, 3) (6, 256, 256, 3)
2 [D loss: 0.06565388292074203 | D Accuracy: 100.0] [G loss: 11.684367179870605]
[756  64  38  25 647 361]
epoch: images_shape: gen_images.shape: = 3 (6, 256, 256, 3) (6, 256, 256, 3)
3 [D loss: 1.8225406408309937 | D Accuracy: 66.66666865348816] [G loss: 11.193977355957031]
[406 891 107 952 934 558]
epoch: images_shape: gen_images.shape: = 4 (6, 256, 256, 3) (6, 256, 256, 3)
4 [D loss: 0.5780744552612305 | D Accuracy: 75.0] [G loss: 7.683618545532227]
[689 150 147 124 658 215]
epoch: images_shape: gen_images.shape: = 5 (6, 256, 256, 3) (6, 256, 256, 3)
5 [D loss: 0.9996928572654724 | D Accuracy: 91.66666269302368] [G loss: 12.887896537780762]
[951 136  40 831 254 601]
epoch: images_shape: gen_

(25, 256, 256, 3)


...

[180 289 582 277 392 281]
epoch: images_shape: gen_images.shape: = 681 (6, 256, 256, 3) (6, 256, 256, 3)
681 [D loss: 0.36621081829071045 | D Accuracy: 83.33333134651184] [G loss: 8.801403045654297]
[564 198 935 509   7 300]
epoch: images_shape: gen_images.shape: = 682 (6, 256, 256, 3) (6, 256, 256, 3)
682 [D loss: 2.0457634925842285 | D Accuracy: 75.0] [G loss: 6.051755428314209]
[930 741 255 876 678 747]
epoch: images_shape: gen_images.shape: = 683 (6, 256, 256, 3) (6, 256, 256, 3)
683 [D loss: 0.36423590779304504 | D Accuracy: 83.33333730697632] [G loss: 6.555891513824463]
[517 508 452 424 710  72]
epoch: images_shape: gen_images.shape: = 684 (6, 256, 256, 3) (6, 256, 256, 3)
684 [D loss: 1.015965461730957 | D Accuracy: 83.33333730697632] [G loss: 11.41085147857666]
[355 142 556 793 331 218]
epoch: images_shape: gen_images.shape: = 685 (6, 256, 256, 3) (6, 256, 256, 3)
685 [D loss: 0.3144393265247345 | D Accuracy: 91.66666269302368] [G loss: 8.942835807800293]
[690 353 702 352 102  

(25, 256, 256, 3)


[458 196 503 638 926  54]
epoch: images_shape: gen_images.shape: = 701 (6, 256, 256, 3) (6, 256, 256, 3)
701 [D loss: 0.3173205554485321 | D Accuracy: 91.66666269302368] [G loss: 13.463793754577637]
[108  28 845 472 764 303]
epoch: images_shape: gen_images.shape: = 702 (6, 256, 256, 3) (6, 256, 256, 3)
702 [D loss: 0.6608316898345947 | D Accuracy: 83.33333730697632] [G loss: 5.904847621917725]
[484 414 645 552 413 313]
epoch: images_shape: gen_images.shape: = 703 (6, 256, 256, 3) (6, 256, 256, 3)
703 [D loss: 0.08444568514823914 | D Accuracy: 100.0] [G loss: 8.8123779296875]
[132 631 386 499 984 505]
epoch: images_shape: gen_images.shape: = 704 (6, 256, 256, 3) (6, 256, 256, 3)
704 [D loss: 0.9318692684173584 | D Accuracy: 75.0] [G loss: 5.963138580322266]
[330 112 974 359 587 460]
epoch: images_shape: gen_images.shape: = 705 (6, 256, 256, 3) (6, 256, 256, 3)
705 [D loss: 0.6411973834037781 | D Accuracy: 91.66666269302368] [G loss: 6.654073238372803]
[ 57 287 759 393 876 467]
epoch: im

(25, 256, 256, 3)


[325 816 124 732 658  23]
epoch: images_shape: gen_images.shape: = 721 (6, 256, 256, 3) (6, 256, 256, 3)
721 [D loss: 1.5497804880142212 | D Accuracy: 75.0] [G loss: 3.6965465545654297]
[539 386 434 590 639  50]
epoch: images_shape: gen_images.shape: = 722 (6, 256, 256, 3) (6, 256, 256, 3)
722 [D loss: 0.6302688717842102 | D Accuracy: 83.33333730697632] [G loss: 5.537242889404297]
[499 745 133  49 812  41]
epoch: images_shape: gen_images.shape: = 723 (6, 256, 256, 3) (6, 256, 256, 3)
723 [D loss: 2.41992449760437 | D Accuracy: 58.33333730697632] [G loss: 9.761425018310547]
[228 320 421 559 138 741]
epoch: images_shape: gen_images.shape: = 724 (6, 256, 256, 3) (6, 256, 256, 3)
724 [D loss: 2.255138874053955 | D Accuracy: 58.33333730697632] [G loss: 8.585269927978516]
[248 392 132 331 929 762]
epoch: images_shape: gen_images.shape: = 725 (6, 256, 256, 3) (6, 256, 256, 3)
725 [D loss: 0.5503103733062744 | D Accuracy: 83.33333134651184] [G loss: 6.772220134735107]
[817 331 764 228 205 746]

In [0]:
generate_imgs(25,(0.0, 0.1) , "")
